<a href="https://colab.research.google.com/github/Bahramif/Deep-Trading/blob/master/FLIPP_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-cloud-vision

     |████████████████████████████████| 419kB 4.9MB/s 


In [0]:
!pip install requests


In [1]:
from google.colab import files
uploaded = files.upload()

Saving My First Project-071ea935f8a8.json to My First Project-071ea935f8a8.json


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My First Project-071ea935f8a8.json"

In [0]:
Meat=['meat','fish','salmon','ground beef','steak','sirloin','ham','trout']
Grains_bread=['pasta','rice','bread']
oil=['butter','oil']
Diary=['Egg','cheese','yogurt','milk']
Produce=['onion','garlic']


fruits=['apple', 'apricot','banana','blackberry','cherry','grapefruit','mango','kiwi','orange','papaya','passion fruit'
,'peach','pear','pea','pineapple','plum','watermelon','raspberry','blueberry','grape']

Vegetables=['asparagus','avocado','bean','broccoli','corn','cucumber','carrot','eggplant','lemon','lettuce','mushroom','potatoe','tomato','green pepper']
All=fruits+Vegetables+Produce+Diary+oil+Grains_bread+Meat

## Scrape Flipp

In [0]:
import requests
import pandas as pd

BASE_URL = 'https://flipp.com'
BACKEND_URL = 'https://backflipp.wishabi.com/flipp'
SEARCH_URL = '%s/items/search' % BACKEND_URL
ITEM_URL = '%s/items/' % BACKEND_URL

# def scrape_item(item_id):
#     return requests.get(
#         "%s/%s" % (ITEM_URL, item_id,)
#     ).json()

def search(query, postal_code, locale):
    data = requests.get(
        SEARCH_URL,
        params = {
            'q': query,
            'postal_code': postal_code,
            'locale':locale
        }
    ).json()

    return [
#             scrape_item(x.get('flyer_item_id'))

            x for x in data.get('items')
    ]




In [0]:
# testing search function
search_result = search('Milk','l6a3r7', 'en-ca')
columns = list(search_result[0].keys())


Convert the product list to dataframe

In [0]:

query_results=list()
df=pd.DataFrame()
for items in All:
  try:
    final_dict = {key:[] for key in columns}
    keys = final_dict.keys()
    query_results=search(items,'l6a3r7', 'en-ca')
    for q in query_results:
       for key in keys:
          final_dict[key].append(q[key])
    final_dict2=pd.DataFrame(final_dict)
    df=df.append(final_dict2)
    row=final_dict2.shape[0]
    df.index=range(df.shape[0])
    df.loc[df.shape[0]-row:,'product']=items
  except:
    continue

In [0]:
df_sort=df.sort_values(['merchant_name'])
mechant_name=df_sort.merchant_name.unique()
df_sort["current_price"] = df_sort["current_price"].apply(pd.to_numeric)
df_sort2=df_sort[df_sort.current_price>0]

Select the grocery sores from the merchant list

In [0]:
food_store=[]
for x in mechant_name:
  if 'food' in x.lower():
    food_store.append(x)
  if 'supermarket' in x.lower():
    food_store.append(x)
  if 'superstore' in x.lower():
    food_store.append(x)
  if 'mart' in x.lower():
    food_store.append(x)
  if 'fresh' in x.lower():
    food_store.append(x)
  if x in ['Walmart','Zehrs','Starsky','Sobeys','Price Chopper','No Frills','Longos','Metro','Loblaws','Healthy Planet','Giant Tiger']:
    food_store.append(x)
    

In [0]:
df_sort_food=df_sort2[df_sort2.merchant_name.isin(food_store)]
df_sort_food.index=range(df_sort_food.shape[0])
df_sort_food2=df_sort_food

##Image processing and create labels for images

In [0]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate

import io
import requests
from PIL import Image
import matplotlib.pyplot as plt  
def Labeled_image(url):
  data = requests.get(url).content
  image = types.Image(content=data)
  img = Image.open(io.BytesIO(data))
  response = client.label_detection(image=image)
  labels = response.label_annotations
  Label_img=list()
  #   plt.imshow(img)
  #   plt.show()
  for label in labels:
    Label_img.append(label.description)
  return Label_img

df_sort_food2['labels']=df_sort_food2['clean_image_url'].apply(lambda x:Labeled_image(str(x)))

Filter for relevant photos

In [0]:
for i in df_sort_food2.index:
  df_sort_food2.loc[i,'Relevant']=pd.Series(df_sort_food2.loc[i,'labels']).isin(['Fruit','Fish','Meat','Vegetable','Ingredient','Dairy','Oil']).any()


In [0]:
df_sort_food2=df_sort_food2[df_sort_food2.Relevant==True]
# df_sort_food2.to_pickle('./flipp.pkl')

In [6]:
from google.colab import files
uploaded = files.upload()

Saving flipp_product.csv to flipp_product.csv


In [0]:
import pandas as pd
df_sort_food2=pd.read_csv('flipp_product.csv')

##OCR flyers for weight analysis

In [0]:
def detect_text_uri(uri):
  try:
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
#     print('Texts:')
    return texts[0].description
  except:
    return 'error'
  

for i in df_sort_food2.index:
  if df_sort_food2.loc[i,"Relevant"]==True:
    df_sort_food2.loc[i,"ocr_name"]=detect_text_uri(df_sort_food2.loc[i,"clean_image_url"])


Extract weight numbers from ocr text

In [0]:
import re
import inflect
p = inflect.engine()
def Weight(name,product):
  pattern1 = re.compile(r'((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b))' , re.IGNORECASE)
  pattern2 = re.compile(r'(/*str(p.plural(product))'
                       +r'/*?:|'  +str(p.singular_noun(product))
                       +r'/*?:|' + str(product) +r')', re.IGNORECASE)
 
  pattern3 = re.compile(r'('+str(p.singular_noun(product))
   +r'|' + str(product)+r'|((?:[0-9]*\s*Pack)'
    r'|(?:[0-9.]+/*\s*/*(?:g|G|KG|kG|Kg|K9|k9|kg|ML|mL|LB|lb|lB|Lb|L|oz)\b)))'
                       , re.IGNORECASE)
  return (re.findall(pattern1, name),re.findall(pattern2, name),re.findall(pattern3, name))